<a href="https://colab.research.google.com/github/stephkariuki19/final-project-colabs/blob/main/projectIterOne.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
DISEASES:
THE DISEASES:
Pneumonia
Consolidation
Infiltration
Pneumothorax
Asthma
Emphysema
Fibrosis
Covid- 19
 Effusion
Atelectasis
Pleural_thickening
Cardiomegaly
Nodule Mass
TB
'''

Using langchain to load data from trusted sources

In [ ]:
!pip install --upgrade --quiet  langchain-pinecone langchain-openai langchain

In [ ]:
pip install langchain-community

In [ ]:
from langchain_community.document_loaders import AsyncHtmlLoader

urls = ["https://www.mayoclinic.org/diseases-conditions/enlarged-heart/symptoms-causes/syc-20355436"]
loader = AsyncHtmlLoader(urls)
docs = loader.load()




Fetching pages: 100%|##########| 1/1 [00:00<00:00,  2.35it/s]


In [ ]:
pip install html2text

In [ ]:
#function to output results of webpage only
from langchain_community.document_transformers import Html2TextTransformer

# Define the topics of interest
topics = ['symptoms', 'when to see a doctor', 'causes', 'risk factors', 'complications', 'prevention', 'vaccination']

html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)

# Dictionary to store content for each topic
topic_content = {topic: [] for topic in topics}

# Iterate over transformed documents
for doc in docs_transformed:
    # Iterate over each topic
    for topic in topics:
        # Find the index of the topic in the page content
        index = doc.page_content.lower().find(topic)
        if index != -1:
            # Find the end index of the topic's section
            next_topic_index = len(doc.page_content)
            for next_topic in topics:
                next_index = doc.page_content.lower().find(next_topic)
                if next_index != -1 and next_index > index:
                    next_topic_index = min(next_topic_index, next_index)
            # Extract content for the topic
            topic_content[topic].append(doc.page_content[index:next_topic_index])

# Print content for each topic
for topic, content_list in topic_content.items():
    print(f"Content for topic '{topic}':")
    for content in content_list:
        print(content)
        print("---------------------")


In [ ]:
#function to output results of webpage and save to my Google drive
from google.colab import drive
from langchain_community.document_transformers import Html2TextTransformer

# Mount Google Drive
drive.mount('/content/drive',force_remount=True)

# Define the topics of interest
topics = ['symptoms', 'when to see a doctor', 'causes', 'risk factors', 'complications', 'prevention', 'vaccination']

html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)

# Dictionary to store content for each topic
topic_content = {topic: [] for topic in topics}

# Iterate over transformed documents
for doc in docs_transformed:
    # Iterate over each topic
    for topic in topics:
        # Find the index of the topic in the page content
        index = doc.page_content.lower().find(topic)
        if index != -1:
            # Find the end index of the topic's section
            next_topic_index = len(doc.page_content)
            for next_topic in topics:
                next_index = doc.page_content.lower().find(next_topic)
                if next_index != -1 and next_index > index:
                    next_topic_index = min(next_topic_index, next_index)
            # Extract content for the topic
            topic_content[topic].append(doc.page_content[index:next_topic_index])

# Write content for each topic to pneumonia.txt in Google Drive
file_path = '/content/drive/My Drive/Cardiomegaly.txt'
with open(file_path, 'w', encoding='utf-8') as file:
    for topic, content_list in topic_content.items():
        file.write(f"Content for topic '{topic}':\n")
        for content in content_list:
            file.write(content + "\n")
            file.write("---------------------\n")

print(f"Content written to {file_path} successfully.")


Mounted at /content/drive
Content written to /content/drive/My Drive/Cardiomegaly.txt successfully.


loading data to a vector store

In [ ]:
from langchain_community.document_loaders import TextLoader

In [ ]:
pip install qdrant-client

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance

qdrant_client = QdrantClient(
    'https://d587747f-8acf-40ea-a289-8d8050de9a3f.us-east4-0.gcp.cloud.qdrant.io:6333',
    api_key="orEjosb4oaqYDh5AGOqAd1Ab4Iyj1UR7vVgheTDIIL-JhE6kREt0LQ",
)

In [ ]:
qdrant_client.recreate_collection(
    collection_name='diseases',
    vectors_config=VectorParams(size=384, distance=Distance.COSINE),
)

True

In [ ]:
from google.colab import files

# Upload the zip file containing the folder
uploaded = files.upload()


In [ ]:
import os
import numpy as np
import json

# Specify the folder path containing text files
folder_path = r'/content/dataFolder'

# Initialize lists to store startup data and vectors
startup_data = []
vectors = []

# Iterate over each file in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.txt'):
        # Load text data from the current file
        with open(os.path.join(folder_path, file_name), 'r') as file:
            content = file.read()
            # Check if the text file contains JSON data
            try:
                data = json.loads(content)
                startup_data.append(data)
            except json.JSONDecodeError:
                # If not JSON, assume it contains vectors (e.g., numpy arrays)
                vector = np.fromstring(content, dtype=float, sep=' ')  # Adjust dtype and sep as needed
                vectors.append(vector)

# Convert vectors list to a single numpy array
vectors = np.stack(vectors)

# Now you can use startup_data and vectors as needed


<ipython-input-18-a2e2e2f255d7>:24: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  vector = np.fromstring(content, dtype=float, sep=' ')  # Adjust dtype and sep as needed


In [ ]:
qdrant_client.upload_collection(
    collection_name='diseases',
    vectors=vectors,
    ids=None,  # Vector ids will be assigned automatically
    batch_size=256  # How many vectors will be uploaded in a single request?
)

In [ ]:
pip install qdrant-client[fastembed]

In [ ]:
qdrant_client.recreate_collection(
    collection_name='my_collection',
    vectors_config=VectorParams(size=384, distance=Distance.COSINE),
)

True

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from fastembed.embedding import DefaultEmbedding
import numpy as np
import os

embedding_model = DefaultEmbedding()

folder_path = r'/content/dataFolder'
for file_name in os.listdir(folder_path):
  loader = TextLoader(file_name)
  documents = loader.load()
  text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)  #might change chunk_overlap
  docs = text_splitter.split_documents(documents)


loader = TextLoader("potato context.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
embeddings: List[np.ndarray] = list(embedding_model.embed(docs))